In [ ]:
# coding: UTF-8
import argparse
import logging
import os
import random
from datetime import datetime
from posixpath import join
import sys 
sys.path.append("..") 
import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *

from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *

### 代码组织结构
DBP15KRawNeighbors():  

load读取的是labse编码的entity的embedding    

In [ ]:
with open('E:/ForkWork/SelfKG/data/DBP15K/zh_en/LaBSE_emb_1.pkl', 'rb') as f:
    id_entity = pickle.load(f)

In [ ]:
print("the length of id_entity is {}, type is {}".format(len(id_entity),type(id_entity)))
print(len(id_entity[0][0]))
x = np.array(id_entity[0])
x.shape

id_neighbors_loader读取的是src/tgt的triple(ids)。这里判断tail是head的邻居.
1. head_str其实是row[head]的embedding  
2. id_neighbors_dict保存的是key:head; value:head的所有tail的embedding
和 key:tail; value:tail的所有head的embedding。保存了所有实体id对应的邻居embedding，第0个代表的是自己。

In [ ]:
filepath = 'E:/ForkWork/SelfKG/data/DBP15K/zh_en/triples_1'
data = pd.read_csv(filepath, header=None, sep='\t')
data.columns = ['head', 'relation', 'tail']
id_neighbors_dict = {}
for index, row in data.iterrows():
    head_str = id_entity[int(row['head'])][0]
    tail_str = id_entity[int(row['tail'])][0]

    if not int(row['head']) in id_neighbors_dict.keys():
        id_neighbors_dict[int(row['head'])] = [head_str]
    if not tail_str in id_neighbors_dict[int(row['head'])]:
        id_neighbors_dict[int(row['head'])].append(tail_str)
    
    if not int(row['tail']) in id_neighbors_dict.keys():
        id_neighbors_dict[int(row['tail'])] = [tail_str]
    if not head_str in id_neighbors_dict[int(row['tail'])]:
        id_neighbors_dict[int(row['tail'])].append(head_str)
# len(id_neighbors_dict)

get_adj是创建邻接矩阵，对角，第一行，第一列。每个节点能到自己，第0个表示自己，能到自己的所有邻居，所有邻居能到第0个节点。  

In [ ]:
def get_adj(valid_len):
    adj = torch.zeros(NEIGHBOR_SIZE, NEIGHBOR_SIZE).bool()
    for i in range(0, valid_len):
        adj[i, i] = 1
        adj[0, i] = 1
        adj[i, 0] = 1
    return adj

get_center_adj是造出来：
1. id_adj_tensor_dict：dict，是当前实体的邻接矩阵。20*20  
2. id_neighbors_dict:dict, padding扩展到20个neighbors。v是个二维矩阵. 20*768
比20大的截断

In [ ]:
id_adj_tensor_dict = {}
for k, v in id_neighbors_dict.items():
    if len(v) < NEIGHBOR_SIZE:
        id_adj_tensor_dict[k] = get_adj(len(v))
        id_neighbors_dict[k] = v + [[0]*LaBSE_DIM] * (NEIGHBOR_SIZE - len(v))
    else:
        id_adj_tensor_dict[k] = get_adj(NEIGHBOR_SIZE)
        id_neighbors_dict[k] = v[:NEIGHBOR_SIZE]


In [ ]:
print(len(id_adj_tensor_dict))
print(len(id_neighbors_dict))
#entity一共19388个

In [ ]:
for k in id_adj_tensor_dict:
    print(k)

In [ ]:
x_train_adj = None
x_train = []
y_train = []
for k in id_neighbors_dict:
    if x_train_adj==None:
        x_train_adj = id_adj_tensor_dict[k].unsqueeze(0)
    else:
        x_train_adj = torch.cat((x_train_adj,id_adj_tensor_dict[k].unsqueeze(0)),dim=0)
    x_train.append(id_neighbors_dict[k])
    y_train.append([k])

In [ ]:
x_train_adj.shape
# 7w+的triple 19388个entity

In [ ]:
x_train = torch.Tensor(x_train)
x_train = torch.cat((x_train,x_train_adj),dim=2)
y_train = torch.Tensor(y_train).long()
print(x_train.shape)
print(y_train.shape)